# 加载库函数

In [1]:
import numpy as np
import scipy.sparse as sp
import networkx as nx
import networkx.algorithms.isomorphism as iso
import itertools
import os.path
import subprocess
import torch.nn.functional as F
import pickle
import os
import scipy.sparse as sp
import torch
import json
import scipy.io as sio
from torch.nn import Embedding
#import scipy.sparse as spsparse
from torch.nn import Parameter
import torch
import torch.nn as nn
from scipy.sparse import csr_matrix
import torch.nn.functional as F
import argparse
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score

# 加载数据，数据处理

In [2]:
"""
Disclaimer: functions defined from lines 15 to 36 in this file come from 
tkipf/gae original repository on Graph Autoencoders. Moreover, the
mask_test_edges_general_link_prediction function is borrowed from 
philipjackson's mask_test_edges pull request on this same repository.
"""

def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

# 任务一

In [3]:
def mask_test_edges_general_link_prediction(adj, test_percent=10., val_percent=5.):
    """
    Task 1: General Directed Link Prediction: get Train/Validation/Test

    :param adj: complete sparse adjacency matrix of the graph
    :param test_percent: percentage of edges in test set
    :param val_percent: percentage of edges in validation set
    :return: train incomplete adjacency matrix, validation and test sets
    """

    # Remove diagonal elements of adjacency matrix
    adj = adj - sp.dia_matrix((adj.diagonal()[None, :], [0]), shape = adj.shape)
    adj.eliminate_zeros()
    edges_positive, _, _ = sparse_to_tuple(adj)

    # Number of positive (and negative) edges in test and val sets:
    num_test = int(np.floor(edges_positive.shape[0] / (100. / test_percent)))
    num_val = int(np.floor(edges_positive.shape[0] / (100. / val_percent)))

    # Sample positive edges for test and val sets:
    edges_positive_idx = np.arange(edges_positive.shape[0])
    np.random.shuffle(edges_positive_idx)
    val_edge_idx = edges_positive_idx[:num_val]
    # positive val edges
    val_edges = edges_positive[val_edge_idx]
    test_edge_idx = edges_positive_idx[num_val:(num_val + num_test)]
    # positive test edges
    test_edges = edges_positive[test_edge_idx]
    # positive train edges
    train_edges = np.delete(edges_positive, np.hstack([test_edge_idx, val_edge_idx]), axis = 0)

    # (Text from philipjackson)
    # The above strategy for sampling without replacement will not work for sampling
    # negative edges on large graphs, because the pool of negative edges
    # is much much larger due to sparsity, therefore we'll use the following strategy:
    # 1. sample random linear indices from adjacency matrix WITH REPLACEMENT
    # (without replacement is super slow). sample more than we need so we'll probably
    # have enough after all the filtering steps.
    # 2. remove any edges that have already been added to the other edge lists
    # 3. convert to (i,j) coordinates
    # 4. remove any duplicate elements if there are any
    # 5. remove any diagonal elements
    # 6. if we don't have enough edges, repeat this process until we get enough
    positive_idx, _, _ = sparse_to_tuple(adj) # [i,j] coord pairs for all true edges
    positive_idx = positive_idx[:,0]*adj.shape[0] + positive_idx[:,1] # linear indices
    # Test set
    test_edges_false = np.empty((0,2),dtype='int64')
    idx_test_edges_false = np.empty((0,),dtype='int64')
    while len(test_edges_false) < len(test_edges):
        # step 1:
        idx = np.random.choice(adj.shape[0]**2, 2*(num_test - len(test_edges_false)), replace = True)
        # step 2:
        idx = idx[~np.in1d(idx, positive_idx, assume_unique = True)]
        idx = idx[~np.in1d(idx, idx_test_edges_false, assume_unique = True)]
        # step 3:
        rowidx = idx // adj.shape[0]
        colidx = idx % adj.shape[0]
        coords = np.vstack((rowidx, colidx)).transpose()
        # step 4:
        coords = np.unique(coords, axis=0)
        np.random.shuffle(coords)
        # step 5:
        coords = coords[coords[:,0] != coords[:,1]]
        # step 6:
        coords = coords[:min(num_test, len(idx))]
        test_edges_false = np.append(test_edges_false, coords, axis = 0)
        idx = idx[:min(num_test, len(idx))]
        idx_test_edges_false = np.append(idx_test_edges_false, idx)

    # Validation set
    val_edges_false = np.empty((0,2), dtype = 'int64')
    idx_val_edges_false = np.empty((0,), dtype = 'int64')
    while len(val_edges_false) < len(val_edges):
        # step 1:
        idx = np.random.choice(adj.shape[0]**2, 2*(num_val - len(val_edges_false)), replace = True)
        # step 2:
        idx = idx[~np.in1d(idx, positive_idx, assume_unique = True)]
        idx = idx[~np.in1d(idx, idx_test_edges_false, assume_unique = True)]
        idx = idx[~np.in1d(idx, idx_val_edges_false, assume_unique = True)]
        # step 3:
        rowidx = idx // adj.shape[0]
        colidx = idx % adj.shape[0]
        coords = np.vstack((rowidx, colidx)).transpose()
        # step 4:
        coords = np.unique(coords, axis = 0)
        np.random.shuffle(coords)
        # step 5:
        coords = coords[coords[:,0] != coords[:,1]]
        # step 6:
        coords = coords[:min(num_val, len(idx))]
        val_edges_false = np.append(val_edges_false, coords, axis=0)
        idx = idx[:min(num_val, len(idx))]
        idx_val_edges_false = np.append(idx_val_edges_false, idx)

    # Sanity checks:
    train_edges_linear = train_edges[:,0]*adj.shape[0] + train_edges[:,1]
    test_edges_linear = test_edges[:,0]*adj.shape[0] + test_edges[:,1]
    assert not np.any(np.in1d(idx_test_edges_false, positive_idx))
    assert not np.any(np.in1d(idx_val_edges_false, positive_idx))
    assert not np.any(np.in1d(val_edges[:,0]*adj.shape[0] + val_edges[:,1], train_edges_linear))
    assert not np.any(np.in1d(test_edges_linear, train_edges_linear))
    assert not np.any(np.in1d(val_edges[:,0]*adj.shape[0] + val_edges[:,1], test_edges_linear))

    # Re-build train adjacency matrix
    #data = np.ones(train_edges.shape[0])
    #adj_train = sp.csr_matrix((data, (train_edges[:, 0], train_edges[:, 1])), shape = adj.shape)

    return train_edges, val_edges, val_edges_false, test_edges, test_edges_false

# 任务二

In [4]:
def mask_test_edges_biased_negative_samples(adj, test_percent=10.):
    """
    Task 2: General Biased Negative Samples (B.N.S.) Directed Link
    Prediction: get Train and Test sets

    :param adj: complete sparse adjacency matrix of the graph
    :param test_percent: percentage of edges in test set
    :return: train incomplete adjacency matrix and test sets
    """

    # Remove diagonal elements of adjacency matrix
    adj = adj - sp.dia_matrix((adj.diagonal()[None, :], [0]), shape = adj.shape)
    adj.eliminate_zeros()
    val_edges, val_edges_false, test_edges, test_edges_false = None, None, None, None

    ## Retrieve all unidirectional edges
    adj_sym = (adj + adj.T).sign()
    adj_tilde = (adj_sym - adj).T
    adj_tilde.eliminate_zeros()
    edges_positive, _, _ = sparse_to_tuple(adj_tilde)

    # Number of positive (= to number of negative) test node pairs to sample
    num_test = int(np.floor(edges_positive.shape[0] / (100. / test_percent)))

    # Sampling of positive node pairs
    edges_idx = np.arange(edges_positive.shape[0])
    np.random.shuffle(edges_idx)
    test_edges_idx = edges_idx[:num_test]
    test_edges = edges_positive[test_edges_idx]

    # In this setting, the reverse node pairs constitute negative samples
    test_edges_false = np.fliplr(test_edges)

    # Get training incomplete adjacency matrix
    train_edges = np.delete(edges_positive, np.hstack([test_edges_idx]), axis = 0)
    #data = np.ones(train_edges.shape[0])
    #adj_train = sp.csr_matrix((data, (train_edges[:, 0], train_edges[:, 1])), shape = adj.shape)

    # Validation set: not implemented for Task 2
    # therefore, val_edges and val_edges_false are None
    return train_edges, val_edges, val_edges_false, test_edges, test_edges_false

# 任务三

In [5]:
# Task 3 - Bidirectional Link Prediction
def mask_test_edges_bidirectional_link_prediction(adj):
    """
    Task 3: Bidirectionality Prediction: get Train and Test sets

    :param adj: complete sparse adjacency matrix of the graph
    :return: train incomplete adjacency matrix and test sets
    """

    # Remove diagonal elements
    adj = adj - sp.dia_matrix((adj.diagonal()[None, :], [0]), shape = adj.shape)
    adj.eliminate_zeros()
    val_edges, val_edges_false, test_edges, test_edges_false = None, None, None, None

    ## Unidirectional edges
    adj_sym = (adj + adj.T).sign()
    adj_tilde = (adj_sym - adj).T
    adj_tilde.eliminate_zeros()

    ## Bidirectional edges (they usually are few, so they are all in test set)
    adj_sym_ones = adj - adj_tilde
    adj_sym_ones.eliminate_zeros()

    # Positive node pairs in test set (bidirectional edges)
    test_edges, _, _ = sparse_to_tuple(adj_sym_ones)
    test_edges = test_edges[test_edges[:,1] > test_edges[:,0],:]

    # Negative node pairs in test set (unidirectional edges)
    test_edges_false, _, _ = sparse_to_tuple(adj_tilde)
    test_edges_false = test_edges_false[test_edges_false[:,0] > test_edges_false[:,1],:]
    test_edges_false = np.fliplr(test_edges_false)
    # Sampling of negative node pairs
    edges_negative_idx = np.arange(test_edges_false.shape[0])
    np.random.shuffle(edges_negative_idx)
    test_edges_false_idx = edges_negative_idx[:test_edges.shape[0]]
    test_edges_false = test_edges_false[test_edges_false_idx]

    # Get training incomplete adjacency matrix
    # 1 of the 2 directions of each bidirectional edge is masked
    adj_train = (adj - sp.triu(adj_sym_ones))

    # Validation set: not implemented for Task 2
    # val_edges and val_edges_false are None
    return adj_train, val_edges, val_edges_false, test_edges, test_edges_false

In [6]:
def load_data(dataset):
    """ Load datasets from text files

    :param dataset: 'cora', 'citeseer' or 'google' graph dataset.
    :return: n*n sparse adjacency matrix and n*f node-level feature matrix

    Note: in this paper, all three datasets are assumed to be featureless.
    As a consequence, the feature matrix is the identity matrix I_n.
    """
    if dataset == 'cora':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/cora.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Cora raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset == 'citeseer':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/citeseer.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Citeseer raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset == 'google':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/GoogleNw.txt",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        features = sp.identity(adj.shape[0])

    else:
        raise ValueError('Undefined dataset!')

    return adj, features

In [7]:
def get_graph(DATASET='cora', task=1):
    adj_init, features = load_data(DATASET)
    if task==1:
        train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_general_link_prediction(adj_init, 10, 5)
    elif task==2:
        train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_biased_negative_samples(adj_init)
    else:
        train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_bidirectional_link_prediction(adj_init)
    
    all_negative_edges = np.argwhere(adj_init.toarray()==0)
    
    if task==2 or task==3:
        all_train_edges_false = np.delete(all_negative_edges, test_edges_false, axis = 0)
    else:
        all_train_edges_false = np.delete(all_negative_edges, np.vstack([test_edges_false, val_edges_false]), axis = 0)
    
    if DATASET=='cora':
        nodes_number = 2708
    elif DATASET=='citeseer':
        nodes_number = 3327
    
    ################## 处理测试集  #############################
    test_posi_mask = [test_edges[i][0]*nodes_number+test_edges[i][1] for i in range(len(test_edges))]
    test_false_mask = [test_edges_false[i][0]*nodes_number+test_edges_false[i][1] for i in range(len(test_edges_false))]
    test_mask = test_posi_mask + test_false_mask
    test_mask = torch.tensor(test_mask)
    test_edge_true_or_false = torch.zeros(len(test_mask))
    test_edge_true_or_false[:len(test_posi_mask)] = 1
    
    ############### 处理训练集  ################################
    negative_index = np.random.choice(range(all_train_edges_false.shape[0]), train_edges.shape[0]*4, replace=False)
    train_edges_false = all_train_edges_false[negative_index]
    train_posi_mask = [train_edges[i][0]*nodes_number+train_edges[i][1] for i in range(len(train_edges))]
    train_false_mask = [train_edges_false[i][0]*nodes_number+train_edges_false[i][1] for i in range(len(train_edges_false))]
    train_mask = train_posi_mask + train_false_mask
    train_mask = torch.tensor(train_mask)
    #train_mask = torch.tensor(train_posi_mask)
    train_edge_true_or_false = torch.zeros(len(train_mask))
    train_edge_true_or_false[:len(train_posi_mask)] = 1
    
    return train_edge_true_or_false, test_edge_true_or_false, train_mask.long(), test_mask.long(), nodes_number

In [8]:
#train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(task=2)

# 定义模型

In [9]:
def predict_acc(recons_edges, true_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    ap = average_precision_score(true_edges, predict_edges)
    auc = roc_auc_score(true_edges, predict_edges)
    print("AP SCORE： ", ap)
    print("AUC SCORE: ", auc)
    return ap, auc

################  更新节点内容  #######################################
class reconstruction_graph(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, context_embedding):
        super(reconstruction_graph, self).__init__()
        self.context_embedding = Parameter(context_embedding)
        
    def forward(self, importance_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(self.context_embedding[:, None]-self.context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((self.context_embedding.unsqueeze(1) - importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        #return 2*sigmoid_modle(nodes_similar)-1
        return torch.exp(-nodes_similar)
        
    
class update_nodes_embedding(nn.Module):
    def __init__(self, context_embedding):
        super(update_nodes_embedding, self).__init__()
        self.reconstruction_module = reconstruction_graph(context_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, importance_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(importance_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.context_embedding.detach(), ap, auc

####################  更新节点重要性  ######################################
class reconstruction_graph_importance(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, importance_embedding):
        super(reconstruction_graph_importance, self).__init__()
        self.importance_embedding = Parameter(importance_embedding)
        
    def forward(self, context_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(context_embedding[:, None]-context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((context_embedding.unsqueeze(1) - self.importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        #return 2*sigmoid_modle(nodes_similar)-1
        return torch.exp(-nodes_similar)

class update_nodes_embedding_importance(nn.Module):
    def __init__(self, importance_embedding):
        super(update_nodes_embedding_importance, self).__init__()
        self.reconstruction_module = reconstruction_graph_importance(importance_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, context_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(context_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.importance_embedding.detach(), ap, auc

In [10]:
context_embedding_dim = 12
importance_embedding_dim = 48
beta = 5.2
ALPHA = 1.0
LR = 0.01
MOMENTUM = 0.9
EPOCH = 5
N = 25.0    # 目前25最佳
sigmoid_modle = torch.nn.Sigmoid()

dataset = 'cora'

In [11]:
all_ap_first_best_auc = []
all_ap_first_best_ap = []

all_auc_first_best_auc = []
all_auc_first_best_ap = []

for i in range(10):

    train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(DATASET=dataset, task=2)
    context_embedding = Embedding(nodes_number, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_context_embeddings = context_embedding.weight.detach()

    importance_embedding = Embedding(importance_embedding_dim, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_importance_embeddings = importance_embedding.weight.detach()
    
    auc_first_best_auc = 0
    auc_first_best_ap = 0
    ap_first_best_auc = 0
    ap_first_best_ap = 0

    for module_epoch in range(100):
        print("######################### 模块循环 ： %d ##########################"%module_epoch)
        update_nodes_module = update_nodes_embedding(nodes_context_embeddings)
        nodes_context_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_importance_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
        update_nodes_importance_module = update_nodes_embedding_importance(nodes_importance_embeddings)
        nodes_importance_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_context_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
            
    all_ap_first_best_ap.append(ap_first_best_ap)
    all_ap_first_best_auc.append(ap_first_best_auc)
    
    all_auc_first_best_ap.append(auc_first_best_ap)
    all_auc_first_best_auc.append(auc_first_best_auc)

######################### 模块循环 ： 0 ##########################
Epoch: 00, Loss: 4917.6719
Epoch: 01, Loss: 4903.6201
Epoch: 02, Loss: 4877.4492
Epoch: 03, Loss: 4841.6509
Epoch: 04, Loss: 4798.9766
AP SCORE：  0.5487846323515739
AUC SCORE:  0.5450668334960938
Epoch: 00, Loss: 4906.6812
Epoch: 01, Loss: 4855.2158
Epoch: 02, Loss: 4800.6353
Epoch: 03, Loss: 4743.4751
Epoch: 04, Loss: 4684.1924
AP SCORE：  0.644708315820862
AUC SCORE:  0.6125679016113281
######################### 模块循环 ： 1 ##########################
Epoch: 00, Loss: 4623.1904
Epoch: 01, Loss: 4614.3726
Epoch: 02, Loss: 4597.6660
Epoch: 03, Loss: 4573.9443
Epoch: 04, Loss: 4544.0205
AP SCORE：  0.6741256516208669
AUC SCORE:  0.6374702453613281
Epoch: 00, Loss: 4508.6499
Epoch: 01, Loss: 4468.5200
Epoch: 02, Loss: 4424.2583
Epoch: 03, Loss: 4376.4316
Epoch: 04, Loss: 4325.5542
AP SCORE：  0.7080050963921354
AUC SCORE:  0.6695060729980469
######################### 模块循环 ： 2 ##########################
Epoch: 00, Loss: 4272.0869
Epoc

Epoch: 00, Loss: 2137.4912
Epoch: 01, Loss: 2136.8301
Epoch: 02, Loss: 2135.5776
Epoch: 03, Loss: 2133.8005
Epoch: 04, Loss: 2131.5610
AP SCORE：  0.8638739573455294
AUC SCORE:  0.8604278564453125
Epoch: 00, Loss: 2128.9185
Epoch: 01, Loss: 2125.9258
Epoch: 02, Loss: 2122.6345
Epoch: 03, Loss: 2119.0901
Epoch: 04, Loss: 2115.3345
AP SCORE：  0.8648268291743706
AUC SCORE:  0.8619041442871094
######################### 模块循环 ： 19 ##########################
Epoch: 00, Loss: 2111.4062
Epoch: 01, Loss: 2110.8315
Epoch: 02, Loss: 2109.7419
Epoch: 03, Loss: 2108.1963
Epoch: 04, Loss: 2106.2483
AP SCORE：  0.8654521912138913
AUC SCORE:  0.8626937866210938
Epoch: 00, Loss: 2103.9487
Epoch: 01, Loss: 2101.3440
Epoch: 02, Loss: 2098.4783
Epoch: 03, Loss: 2095.3906
Epoch: 04, Loss: 2092.1174
AP SCORE：  0.866489041851491
AUC SCORE:  0.8639602661132812
######################### 模块循环 ： 20 ##########################
Epoch: 00, Loss: 2088.6924
Epoch: 01, Loss: 2088.1902
Epoch: 02, Loss: 2087.2383
Epoch: 03,

Epoch: 01, Loss: 1927.5100
Epoch: 02, Loss: 1927.2889
Epoch: 03, Loss: 1926.9747
Epoch: 04, Loss: 1926.5769
AP SCORE：  0.8770637052979707
AUC SCORE:  0.8755569458007812
Epoch: 00, Loss: 1926.1056
Epoch: 01, Loss: 1925.5682
Epoch: 02, Loss: 1924.9739
Epoch: 03, Loss: 1924.3286
Epoch: 04, Loss: 1923.6392
AP SCORE：  0.8771489406770228
AUC SCORE:  0.8756370544433594
######################### 模块循环 ： 37 ##########################
Epoch: 00, Loss: 1922.9114
Epoch: 01, Loss: 1922.8016
Epoch: 02, Loss: 1922.5935
Epoch: 03, Loss: 1922.2974
Epoch: 04, Loss: 1921.9222
AP SCORE：  0.8772519859615919
AUC SCORE:  0.875762939453125
Epoch: 00, Loss: 1921.4775
Epoch: 01, Loss: 1920.9713
Epoch: 02, Loss: 1920.4105
Epoch: 03, Loss: 1919.8020
Epoch: 04, Loss: 1919.1511
AP SCORE：  0.8773924218487413
AUC SCORE:  0.8758087158203125
######################### 模块循环 ： 38 ##########################
Epoch: 00, Loss: 1918.4644
Epoch: 01, Loss: 1918.3606
Epoch: 02, Loss: 1918.1639
Epoch: 03, Loss: 1917.8840
Epoch: 04,

Epoch: 02, Loss: 1870.1351
Epoch: 03, Loss: 1869.9967
Epoch: 04, Loss: 1869.8210
AP SCORE：  0.8785366543554001
AUC SCORE:  0.8765716552734375
Epoch: 00, Loss: 1869.6133
Epoch: 01, Loss: 1869.3760
Epoch: 02, Loss: 1869.1123
Epoch: 03, Loss: 1868.8258
Epoch: 04, Loss: 1868.5190
AP SCORE：  0.8785262850024752
AUC SCORE:  0.876556396484375
######################### 模块循环 ： 55 ##########################
Epoch: 00, Loss: 1868.1941
Epoch: 01, Loss: 1868.1444
Epoch: 02, Loss: 1868.0507
Epoch: 03, Loss: 1867.9171
Epoch: 04, Loss: 1867.7476
AP SCORE：  0.8785427427188521
AUC SCORE:  0.8765525817871094
Epoch: 00, Loss: 1867.5469
Epoch: 01, Loss: 1867.3174
Epoch: 02, Loss: 1867.0627
Epoch: 03, Loss: 1866.7860
Epoch: 04, Loss: 1866.4895
AP SCORE：  0.8785610317415042
AUC SCORE:  0.8765373229980469
######################### 模块循环 ： 56 ##########################
Epoch: 00, Loss: 1866.1754
Epoch: 01, Loss: 1866.1277
Epoch: 02, Loss: 1866.0369
Epoch: 03, Loss: 1865.9077
Epoch: 04, Loss: 1865.7440
AP SCORE： 

Epoch: 03, Loss: 1840.8353
Epoch: 04, Loss: 1840.7318
AP SCORE：  0.8776867216811198
AUC SCORE:  0.8756942749023438
Epoch: 00, Loss: 1840.6086
Epoch: 01, Loss: 1840.4679
Epoch: 02, Loss: 1840.3116
Epoch: 03, Loss: 1840.1414
Epoch: 04, Loss: 1839.9587
AP SCORE：  0.8776512516960598
AUC SCORE:  0.8756752014160156
######################### 模块循环 ： 73 ##########################
Epoch: 00, Loss: 1839.7656
Epoch: 01, Loss: 1839.7360
Epoch: 02, Loss: 1839.6797
Epoch: 03, Loss: 1839.6002
Epoch: 04, Loss: 1839.4989
AP SCORE：  0.877590695459861
AUC SCORE:  0.8756256103515625
Epoch: 00, Loss: 1839.3787
Epoch: 01, Loss: 1839.2410
Epoch: 02, Loss: 1839.0887
Epoch: 03, Loss: 1838.9227
Epoch: 04, Loss: 1838.7448
AP SCORE：  0.8775882231949503
AUC SCORE:  0.8756256103515625
######################### 模块循环 ： 74 ##########################
Epoch: 00, Loss: 1838.5560
Epoch: 01, Loss: 1838.5270
Epoch: 02, Loss: 1838.4723
Epoch: 03, Loss: 1838.3943
Epoch: 04, Loss: 1838.2954
AP SCORE：  0.8775710953301352
AUC SCO

...

Epoch: 03, Loss: 2037.8291
Epoch: 04, Loss: 2036.6310
AP SCORE：  0.8734701645388621
AUC SCORE:  0.8677520751953125
Epoch: 00, Loss: 2035.2148
Epoch: 01, Loss: 2033.6079
Epoch: 02, Loss: 2031.8364
Epoch: 03, Loss: 2029.9231
Epoch: 04, Loss: 2027.8900
AP SCORE：  0.8740859896586743
AUC SCORE:  0.8685379028320312
######################### 模块循环 ： 24 ##########################
Epoch: 00, Loss: 2025.7562
Epoch: 01, Loss: 2025.4407
Epoch: 02, Loss: 2024.8420
Epoch: 03, Loss: 2023.9919
Epoch: 04, Loss: 2022.9189
AP SCORE：  0.8744362282908108
AUC SCORE:  0.8690109252929688
Epoch: 00, Loss: 2021.6498
Epoch: 01, Loss: 2020.2098
Epoch: 02, Loss: 2018.6211
Epoch: 03, Loss: 2016.9050
Epoch: 04, Loss: 2015.0803
AP SCORE：  0.875051063323859
AUC SCORE:  0.8696327209472656
######################### 模块循环 ： 25 ##########################
Epoch: 00, Loss: 2013.1639
Epoch: 01, Loss: 2012.8796
Epoch: 02, Loss: 2012.3412
Epoch: 03, Loss: 2011.5762
Epoch: 04, Loss: 2010.6100
AP SCORE：  0.8753533104628466
AUC SCO

Epoch: 04, Loss: 1907.3425
AP SCORE：  0.8814519361731197
AUC SCORE:  0.8760719299316406
Epoch: 00, Loss: 1906.9828
Epoch: 01, Loss: 1906.5732
Epoch: 02, Loss: 1906.1193
Epoch: 03, Loss: 1905.6267
Epoch: 04, Loss: 1905.0994
AP SCORE：  0.881520194513418
AUC SCORE:  0.8761062622070312
######################### 模块循环 ： 42 ##########################
Epoch: 00, Loss: 1904.5427
Epoch: 01, Loss: 1904.4580
Epoch: 02, Loss: 1904.2983
Epoch: 03, Loss: 1904.0708
Epoch: 04, Loss: 1903.7828
AP SCORE：  0.8815492479226135
AUC SCORE:  0.8761367797851562
Epoch: 00, Loss: 1903.4413
Epoch: 01, Loss: 1903.0515
Epoch: 02, Loss: 1902.6200
Epoch: 03, Loss: 1902.1512
Epoch: 04, Loss: 1901.6501
AP SCORE：  0.8815390857258123
AUC SCORE:  0.8761482238769531
######################### 模块循环 ： 43 ##########################
Epoch: 00, Loss: 1901.1204
Epoch: 01, Loss: 1901.0400
Epoch: 02, Loss: 1900.8877
Epoch: 03, Loss: 1900.6713
Epoch: 04, Loss: 1900.3972
AP SCORE：  0.8815429767135596
AUC SCORE:  0.8761405944824219
Epo

Epoch: 00, Loss: 1861.7762
Epoch: 01, Loss: 1861.5771
Epoch: 02, Loss: 1861.3560
Epoch: 03, Loss: 1861.1158
Epoch: 04, Loss: 1860.8584
AP SCORE：  0.8821239240675585
AUC SCORE:  0.8759956359863281
######################### 模块循环 ： 60 ##########################
Epoch: 00, Loss: 1860.5861
Epoch: 01, Loss: 1860.5444
Epoch: 02, Loss: 1860.4653
Epoch: 03, Loss: 1860.3531
Epoch: 04, Loss: 1860.2108
AP SCORE：  0.8821324698435822
AUC SCORE:  0.8759803771972656
Epoch: 00, Loss: 1860.0417
Epoch: 01, Loss: 1859.8489
Epoch: 02, Loss: 1859.6349
Epoch: 03, Loss: 1859.4019
Epoch: 04, Loss: 1859.1521
AP SCORE：  0.8821172018286234
AUC SCORE:  0.8759956359863281
######################### 模块循环 ： 61 ##########################
Epoch: 00, Loss: 1858.8879
Epoch: 01, Loss: 1858.8475
Epoch: 02, Loss: 1858.7711
Epoch: 03, Loss: 1858.6622
Epoch: 04, Loss: 1858.5244
AP SCORE：  0.8821034037656623
AUC SCORE:  0.8759803771972656
Epoch: 00, Loss: 1858.3604
Epoch: 01, Loss: 1858.1733
Epoch: 02, Loss: 1857.9657
Epoch: 03

Epoch: 01, Loss: 1836.8245
Epoch: 02, Loss: 1836.6876
Epoch: 03, Loss: 1836.5392
Epoch: 04, Loss: 1836.3798
AP SCORE：  0.8814552575322938
AUC SCORE:  0.8750953674316406
######################### 模块循环 ： 78 ##########################
Epoch: 00, Loss: 1836.2104
Epoch: 01, Loss: 1836.1848
Epoch: 02, Loss: 1836.1357
Epoch: 03, Loss: 1836.0659
Epoch: 04, Loss: 1835.9774
AP SCORE：  0.8814003087069482
AUC SCORE:  0.8750267028808594
Epoch: 00, Loss: 1835.8718
Epoch: 01, Loss: 1835.7520
Epoch: 02, Loss: 1835.6184
Epoch: 03, Loss: 1835.4729
Epoch: 04, Loss: 1835.3169
AP SCORE：  0.8813663665961426
AUC SCORE:  0.8749961853027344
######################### 模块循环 ： 79 ##########################
Epoch: 00, Loss: 1835.1519
Epoch: 01, Loss: 1835.1266
Epoch: 02, Loss: 1835.0785
Epoch: 03, Loss: 1835.0101
Epoch: 04, Loss: 1834.9235
AP SCORE：  0.8813504325077661
AUC SCORE:  0.8749961853027344
Epoch: 00, Loss: 1834.8207
Epoch: 01, Loss: 1834.7030
Epoch: 02, Loss: 1834.5728
Epoch: 03, Loss: 1834.4303
Epoch: 04

Epoch: 02, Loss: 1820.4713
Epoch: 03, Loss: 1820.3676
Epoch: 04, Loss: 1820.2567
AP SCORE：  0.8808364213009665
AUC SCORE:  0.8746147155761719
######################### 模块循环 ： 96 ##########################
Epoch: 00, Loss: 1820.1385
Epoch: 01, Loss: 1820.1204
Epoch: 02, Loss: 1820.0863
Epoch: 03, Loss: 1820.0376
Epoch: 04, Loss: 1819.9756
AP SCORE：  0.8808086483189788
AUC SCORE:  0.8745803833007812
Epoch: 00, Loss: 1819.9021
Epoch: 01, Loss: 1819.8180
Epoch: 02, Loss: 1819.7249
Epoch: 03, Loss: 1819.6229
Epoch: 04, Loss: 1819.5138
AP SCORE：  0.8807813616571771
AUC SCORE:  0.8745574951171875
######################### 模块循环 ： 97 ##########################
Epoch: 00, Loss: 1819.3981
Epoch: 01, Loss: 1819.3801
Epoch: 02, Loss: 1819.3464
Epoch: 03, Loss: 1819.2986
Epoch: 04, Loss: 1819.2379
AP SCORE：  0.8807662778511728
AUC SCORE:  0.8745384216308594
Epoch: 00, Loss: 1819.1655
Epoch: 01, Loss: 1819.0828
Epoch: 02, Loss: 1818.9910
Epoch: 03, Loss: 1818.8911
Epoch: 04, Loss: 1818.7838
AP SCORE：

In [13]:
print("AP FIRST MEAN AP : ", np.array(all_ap_first_best_ap).mean())
print("AP FIRST STD AP : ", np.array(all_ap_first_best_ap).std())

print("AP FIRST MEAN AUC : ", np.array(all_ap_first_best_auc).mean())
print("AP FIRST STD AUC : ", np.array(all_ap_first_best_auc).std())

print("AUC FIRST MEAN AP : ", np.array(auc_first_best_ap).mean())
print("AUC FIRST STD AP : ", np.array(auc_first_best_ap).std())

print("AUC FIRST MEAN AUC : ", np.array(auc_first_best_auc).mean())
print("AUC FIRST STD AUC : ", np.array(auc_first_best_auc).std())

AP FIRST MEAN AP :  0.8926294936491743
AP FIRST STD AP :  0.009970598486289397
AP FIRST MEAN AUC :  0.88297119140625
AP FIRST STD AUC :  0.008059324951636327
AUC FIRST MEAN AP :  0.8816485896971546
AUC FIRST STD AP :  0.0
AUC FIRST MEAN AUC :  0.8762550354003906
AUC FIRST STD AUC :  0.0
